In [3]:
import pandas as pd
import numpy as np

In [4]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return int(x)

In [5]:
def clean_df(df):
    df.duplicated()
    df = df.drop_duplicates()
    df = df.dropna(how='all')
    
    n_cols = []
    for i in range(len(df.columns)):
         n_cols.append(df.columns[i].lower().replace('&','').replace(' ','_'))
    df.columns = n_cols
    df = df.rename(columns={'team__contract':'team_contract'})
    #df.head()
    
    df['weight']= df['weight'].str.replace('lbs','').astype('int')
    #print(fifa_data['weight'])
    
    df['height']= df['height'].str.replace('"','')
    df['height'] = df['height'].apply(lambda x: int(x.split("'")[0])*30.48 + int(x.split("'")[1])*2.54)
    
    df['value'] = df['value'].str.replace('€','')
    df['value'] = df['value'].apply(value_to_float)

    df['wage'] = df['wage'].str.replace('€','')
    df['wage'] = df['wage'].apply(value_to_float)

    df['release_clause'] = df['release_clause'].str.replace('€','')
    df['release_clause'] = df['release_clause'].apply(value_to_float)
    
    df['hits'] = df['hits'].apply(value_to_float)
    cols_to_fill = ['curve','volleys','agility','balance', 'jumping', 'interceptions', 'positioning', 'vision', 'composure', 'sliding_tackle']
    df[cols_to_fill] = df[cols_to_fill].fillna(0)
    #fifa_data.isna().sum()
    
    df['a/w'] = df['a/w'].fillna('Medium')
    df['d/w'] = df['d/w'].fillna('Medium')
    
    df['sm'] = df['sm'].str.rstrip('★').astype('int')
    df['w/f'] = df['w/f'].str.rstrip('★').astype('int')
    df['ir'] = df['ir'].str.rstrip('★').astype('int')
    
    for column in df.iloc[:,df.columns.get_loc('pac'):df.columns.get_loc('hits')]:
        df[column] = df[column].fillna(df[column].mean())
        
    for column in df.iloc[:,df.columns.get_loc('ls'):len(df.columns)-1]:
        #print("Column: ", column)
        df[column] = df[column].apply(lambda x: int(x.split("+")[0]))
        #opt2[column] = fifa_data[column].apply(lambda x: int(x.split("+")[0]) + int(x.split("+")[1]))
    
    return df